<a href="https://colab.research.google.com/github/ishaan27chaturvedi/Multi-Agent-RAG/blob/main/MARAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
pip install -qU langchain-openai

In [11]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = 'XXXXXXXXX'

In [97]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableMap
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def create_agent():
    return (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

def context_identifier(question, agent):
    # This function should identify 5 different contexts based on the question
    return [con_q.split('. ')[1] for con_q in agent.invoke('Create a list of 3 relevant questions that would be required to answer this question. Question - ' + question).split('\n')]

def final_aggregation_agent(question, answers, agent):
    # Aggregate the answers from different agents
    main_ans = agent.invoke(question)
    full_context = "\n".join(answers)
    return agent.invoke('Resumarise this answer- '+main_ans+' to this question - '+question+' by choosing relevant information from the following additional context -'+ full_context)


class MultiAgentRAG:
    def __init__(self, context_identifier, create_agent, final_aggregation_agent):
        self.context_identifier = context_identifier
        self.create_agent = create_agent
        self.final_aggregation_agent = final_aggregation_agent

    def __call__(self, question):
        contexts = self.context_identifier(question, self.create_agent())
        agents = [self.create_agent() for context in contexts]
        answers = [agent.invoke(context) for agent, context in zip(agents, contexts)]
        final_answer = self.final_aggregation_agent(question, answers, self.create_agent())
        return final_answer



In [106]:
question = "What is Memory?"

In [107]:
single_agent = create_agent()
single_agent.invoke(question)

'Memory is the cognitive process of encoding, storing, and retrieving information. It can be categorized into short-term memory (STM) and long-term memory (LTM), with LTM having subtypes of explicit and implicit memory. Short-term memory is limited in capacity and duration, while long-term memory can store information for a long time with essentially unlimited capacity.'

In [108]:
multi_agent_rag = MultiAgentRAG(context_identifier, create_agent, final_aggregation_agent)
multi_agent_rag(question)

'Memory is the mental capacity to store and retrieve information, with sensory memory allowing for retaining sensory impressions briefly. Short-term memory stores information for immediate tasks, while long-term memory stores information for a long time and has two subtypes: explicit (declarative) memory for facts and events, and implicit (procedural) memory for skills and routines.'